# 03 What next?

## Where we are

So far we've identified the course ends that are furthest away from rounds. The original problem was actually posed in reverse: it was to find out how to get to rounds from a particular course end. We can solve this by inverting. If $R$ is one of the rows identified then:

$$R^{-1} . R = I$$

... i.e. if we start at $R^{-1}$ then we reach rounds with the same calling.

In [1]:
from stedman_searching.rows import perm_from_row, row_from_perm

[
    row_from_perm(~perm_from_row(row))
    for row in ['5162473', '5172436', '5162437', '5132476', '5172463', '5132467']
]

['2475136', '2465173', '2465137', '2435176', '2475163', '2435167']

We've missed the original problem specification in some other ways. Here's the original challenge:

> Given a particular *course end* (the row at the end of a slow six), what is the shortest calling that can be used to reach rounds? Which course end (or ends) are furthest from rounds?

We've ignored the fact that rounds can appear as any one of the twelve rows of a six-pair. The course end `2475136` can be brought round at the first row of a quick six using the calling `pppspp-ppppspp` (i.e. within 14 sixes), for example (and it's likely that a shorter solution is available). I'm going to ignore this additional complexity.

The last problem is that we haven't identified the calling required to get between two course ends. This is implicit in the way we have generated the data so let's adapt our system to do this.

## Storing data as we generate rows

Let's create a container that can store additional metadata alongside a row. By overriding Python magic methods we can ignore this metadata when storing the container within a `set` and identifying duplicates.

In [2]:
from stedman_searching.rows import SetRow

set([
    SetRow('123', key='value'),
    SetRow('123', other=42),
    SetRow('321'),
])

{SetRow('123', key='value'), SetRow('321')}

Let's adapt our current algorithm to accumulate the calling so far. We can also store the permutation (as well as the row string) in order to eliminate one conversion step.

In [3]:
from stedman_searching.rows import perm_for_rounds

PAIR_END_TRANSITIONS = [
    (perm_from_row('2467153'), 'pp'),
    (perm_from_row('2463175'), 'p-'),
    (perm_from_row('2463157'), 'ps'),
    (perm_from_row('2456173'), '-p'),
    (perm_from_row('2453167'), '--'),
    (perm_from_row('2457163'), 'sp'),
    (perm_from_row('2453176'), 's-'),
]

def generate_next_row_inner(current_row, pair_end_transition):
    pair_end_perm, pair_end_calling = pair_end_transition

    next_calling = current_row.calling + pair_end_calling
    next_perm = pair_end_perm * current_row.perm

    return SetRow(
        row_from_perm(next_perm),
        calling=next_calling,
        perm=next_perm,
    )

def generate_next_rows(current_rows):
    return set([
        generate_next_row_inner(current_row, pair_end_transition)
        for current_row in current_rows
        for pair_end_transition in PAIR_END_TRANSITIONS
    ])

starting_row = SetRow(
    '1234567',
    calling='',
    perm=perm_for_rounds(6),
)

generate_next_rows(set([starting_row]))

{SetRow('2453167', calling='--', perm=Permutation(6)(0, 1, 3, 2, 4)),
 SetRow('2453176', calling='s-', perm=Permutation(0, 1, 3, 2, 4)(5, 6)),
 SetRow('2456173', calling='-p', perm=Permutation(0, 1, 3, 5, 6, 2, 4)),
 SetRow('2457163', calling='sp', perm=Permutation(0, 1, 3, 6, 2, 4)),
 SetRow('2463157', calling='ps', perm=Permutation(6)(0, 1, 3, 2, 5, 4)),
 SetRow('2463175', calling='p-', perm=Permutation(0, 1, 3, 2, 5, 6, 4)),
 SetRow('2467153', calling='pp', perm=Permutation(0, 1, 3, 6, 2, 5, 4))}

In [4]:
all_rows = set()
rows_per_iteration = [set([starting_row])]

for _ in range(9):
    current_rows = rows_per_iteration[-1]
    next_rows = generate_next_rows(current_rows)
    next_rows = next_rows - all_rows  # prune duplicates
    rows_per_iteration.append(next_rows)
    all_rows = all_rows | next_rows

rows_per_iteration[-1]

{SetRow('5132467', calling='-pp-s-s--psps-sp--', perm=Permutation(6)(0, 4, 3, 1)),
 SetRow('5132476', calling='s-s-ps--pppp-ps-pp', perm=Permutation(0, 4, 3, 1)(5, 6)),
 SetRow('5162437', calling='-pp-s-s--psps-spps', perm=Permutation(6)(0, 4, 3, 1)(2, 5)),
 SetRow('5162473', calling='-pp-s-s--psps-spp-', perm=Permutation(0, 4, 3, 1)(2, 5, 6)),
 SetRow('5172436', calling='s-s-ps--pppp-ps-sp', perm=Permutation(0, 4, 3, 1)(2, 6, 5)),
 SetRow('5172463', calling='s-s-pspssp-ps-p--p', perm=Permutation(0, 4, 3, 1)(2, 6))}

## Working backwards

Since our goal is to reach rounds it seems strange that we're working in the opposite direction. Let's fix that now by inverting each transition permutation and assembling the calling in reverse.

In [5]:
PAIR_END_INVERSE_TRANSITIONS = [
    (~perm, calling)
    for perm, calling in PAIR_END_TRANSITIONS
]

def generate_previous_row_inner(current_row, pair_end_transition):
    pair_end_perm, pair_end_calling = pair_end_transition

    previous_calling = pair_end_calling + current_row.calling  # n.b. order
    previous_perm = pair_end_perm * current_row.perm

    return SetRow(
        row_from_perm(previous_perm),
        calling=previous_calling,
        perm=previous_perm,
    )

def generate_previous_rows(current_rows):
    return set([
        generate_previous_row_inner(current_row, pair_end_transition)
        for current_row in current_rows
        for pair_end_transition in PAIR_END_INVERSE_TRANSITIONS
    ])

all_rows = set()
rows_per_iteration = [set([starting_row])]

for _ in range(9):
    current_rows = rows_per_iteration[-1]
    previous_rows = generate_previous_rows(current_rows)
    previous_rows = previous_rows - all_rows  # prune duplicates
    rows_per_iteration.append(previous_rows)
    all_rows = all_rows | previous_rows

rows_per_iteration[-1]

{SetRow('2435167', calling='--spps-pspppps-pps', perm=Permutation(6)(0, 1, 3, 4)),
 SetRow('2435176', calling='s-spps-pspppps-pps', perm=Permutation(0, 1, 3, 4)(5, 6)),
 SetRow('2465137', calling='pspp-pp-p--ppp-ps-', perm=Permutation(6)(0, 1, 3, 4)(2, 5)),
 SetRow('2465173', calling='p---ps-pppsppp-ps-', perm=Permutation(0, 1, 3, 4)(2, 5, 6)),
 SetRow('2475136', calling='pppp-pp-p--ppp-ps-', perm=Permutation(0, 1, 3, 4)(2, 6, 5)),
 SetRow('2475163', calling='spspps-pspppps-pps', perm=Permutation(0, 1, 3, 4)(2, 6))}

We now have a set (`all_rows`) containing every possible course end on seven bells along with a shortest calling to return us to rounds. This will be the shortest calling but it is likely that other callings will exist that were pruned out while identifying duplicates. We can find a calling for any particular row by looking it up in the set.

Unfortunately this is awkward to do. Python doesn't provide a way to retrieve an item from a set. If you have the item, after all, why would you need to retrieve it from the set? We need to transfer the rows into another data structure in order to look them up.

In [6]:
{ row.row: row.calling for row in all_rows }['6423751']

'-p-p-ps-'